## People Centered COS
## CDOT Data - El Paso County 
## Investigate Bike and Pedestrian Crashes 2020 - 2024
## Visualizations

### Set Up

In [28]:
pip install pandas -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install openpyxl -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install geopandas -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install matplotlib -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install folium -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install branca -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install h3 -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
import matplotlib.pyplot as plt

import folium
from folium import plugins
import matplotlib.cm as cm 
import matplotlib.colors as colors
from branca.element import Element

import h3

### Load Data

In [36]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format
from cdot_clean_geography_columns import handle_missing_values_in_geography_columns, \
fill_missing_city_values, create_truncated_lat_long, combine_loc1_loc2_alphabetically 

In [37]:
# Load CDOT Data for a list of years

years = [2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/"

loaded_data_list = []
for y in years:
    #filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    filename = f"CDOT_ElPaso_Crash_{y}.csv"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_csv(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)

    # some folders are already converted 
    #if y < 2021:
    #    # call conversion code 
    #    year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2020.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(8256, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2021.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10999, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2022.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10222, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2023.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10343, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2024.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(9188, 111)


## Prepare Data

### Limit to Bike and Pedestrian Crashes in El Paso County 

In [38]:
print('Loaded Data Size', cdot_pdf_load.shape)

cdot_pdf_elpaso = cdot_pdf_load.loc[cdot_pdf_load['County'] == 'EL PASO']
print('El Paso County Data Size', cdot_pdf_elpaso.shape)

# bpep - bike, ped el paso
# Harmful events 
# Crash Type 
# Non motorist 
he_list = ['Pedestrian', 'Bicycle/Motorized Bicycle']
tunm_list = ['Pedestrian', 'Bicyclist', 'Other Pedestrian', 'Wheelchair', 'Scooter']
bike_ped_condition = (cdot_pdf_elpaso['First HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Second HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Third HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Fourth HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['MHE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Crash Type'].isin(he_list)) | \
                     (cdot_pdf_elpaso['TU-1 NM Type'].isin(tunm_list)) | \
                     (cdot_pdf_elpaso['TU-2 NM Type'].isin(tunm_list))

cdot_pdf_bpep = cdot_pdf_elpaso.loc[bike_ped_condition]
print('CDOT El Paso bike and ped crashes size', cdot_pdf_bpep.shape)


Loaded Data Size (49008, 143)
El Paso County Data Size (49006, 143)
CDOT El Paso bike and ped crashes size (1230, 143)


### Clean Data

In [39]:
# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_bpep.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)
#display(cdot_pdf_withNans_dedup.head(5))

Data size after drop duplicates (1230, 143)


In [40]:
print(list(cdot_pdf_withNans_dedup.columns))

['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street', 'Crash Date', 'Crash Time', 'Agency Id', 'City', 'County', 'Latitude', 'Longitude', 'Location 1', 'Link', 'Location 2', 'Location', 'Road Description', 'First HE', 'Second HE', 'Third HE', 'Fourth HE', 'MHE', 'Crash Type', 'Approach Overtaking Turn', 'Wild Animal', 'Number Killed', 'Number Injured', 'Injury 00', 'Injury 01', 'Injury 02', 'Injury 03', 'Injury 04', 'Total Vehicles', 'Secondary Crash', 'Construction Zone', 'School Zone', 'Road Contour Curves', 'Road Contour Grade', 'Road Condition', 'Lighting Conditions', 'Weather Condition', 'Weather Condition 2', 'Lane Position', 'TU-1 Direction', 'TU-2 Direction', 'TU-1 Movement', 'TU-2 Movement', 'TU-1 Type', 'TU-2 Type', 'TU-1 Special Function', 'TU-2 Special Function', 'TU-1 Autonomous Vehicle', 'TU-2 Autonomous Vehicle', 'TU-1 Hit And Run', 'TU-2 Hit And Run', 'TU-1 Speed Limit', 'TU-2 Speed Limit', 'TU-1 Estimated Speed', 'TU-2 Estimated Speed', 'TU-1 Speed', 'TU-2 

In [41]:
# Select columns for analysis 
analysis_columns = [
    'CUID',
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Latitude', 
    'Longitude',
    'Location 1', 
    'Location 2',
    'First HE', 
    'Second HE', 
    'Third HE', 
    'Fourth HE', 
    'MHE', 
    'Crash Type', 
    'TU-1 NM Type', 
    'TU-2 NM Type',
    'Number Killed',
]
cdot_pdf = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
print('nan count before handling') 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
print()

# handle nan, null, and other corner cases
cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf)

# check NaN count by column 
print('nan count after handling')
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

nan count before handling
CUID 207
CUID 207
Crash Date 0
Crash Date 0
City 105
City 105
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 10
Latitude 10
Longitude 10
Longitude 10
Location 1 0
Location 1 0
Location 2 4
Location 2 4
First HE 3
First HE 3
Second HE 1087
Second HE 1087
Third HE 1192
Third HE 1192
Fourth HE 1219
Fourth HE 1219
MHE 3
MHE 3
Crash Type 26
Crash Type 26
TU-1 NM Type 707
TU-1 NM Type 707
TU-2 NM Type 554
TU-2 NM Type 554
Number Killed 0
Number Killed 0

nan count after handling
CUID 207
CUID 207
Crash Date 0
Crash Date 0
City 0
City 0
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0
Location 1 0
Location 1 0
Location 2 4
Location 2 4
First HE 3
First HE 3
Second HE 1087
Second HE 1087
Third HE 1192
Third HE 1192
Fourth HE 1219
Fourth HE 1219
MHE 3
MHE 3
Crash Type 26
Crash Type 26
TU-1 NM Type 707
TU-1 NM Type 707
TU-2 NM Type 554
TU-2 NM Type 554
Number Killed 0
Number Killed 0


In [42]:
# Create truncated latitude and longitude for analysis 
cdot_pdf = create_truncated_lat_long(cdot_pdf, 3) 
print(cdot_pdf.columns)

Index(['CUID', 'Crash Date', 'City', 'County', 'Crash Year', 'Latitude',
       'Longitude', 'Location 1', 'Location 2', 'First HE', 'Second HE',
       'Third HE', 'Fourth HE', 'MHE', 'Crash Type', 'TU-1 NM Type',
       'TU-2 NM Type', 'Number Killed', 'Latitude_3dec', 'Longitude_3dec'],
      dtype='object')


In [43]:
# combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************
#cdot_pdf = combine_loc1_loc2_alphabetically(cdot_pdf) 
# check result
#display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

## Play with GeoPandas

In [44]:
gdf = gpd.GeoDataFrame(
    cdot_pdf, 
    geometry=gpd.points_from_xy(
        cdot_pdf['Longitude'],
        cdot_pdf['Latitude']
          )
        )

# have to set the coordinate reference system for map to render
# epsg code - integer or string 
gdf = gdf.set_crs(epsg=4326)
gdf = gdf.to_crs("EPSG:4326")

display(gdf.head())

,CUID,Crash Date,City,County,Crash Year,Latitude,Longitude,Location 1,Location 2,First HE,...,Third HE,Fourth HE,MHE,Crash Type,TU-1 NM Type,TU-2 NM Type,Number Killed,Latitude_3dec,Longitude_3dec,geometry
16,NaN,1/1/2020,COLORADO SPRINGS,EL PASO,2020,38.855290,-104.875720,COLORADO AVE W,32ND ST S,Pedestrian,...,NaN,NaN,Pedestrian,Pedestrian,NaN,Pedestrian,0,38.855,-104.876,POINT (-104.87572 38.85529)
77,NaN,1/3/2020,COLORADO SPRINGS,EL PASO,2020,38.806660,-104.823146,SOUTHGATE RD,NEVADA AVE S,Pedestrian,...,NaN,NaN,Pedestrian,Pedestrian,Pedestrian,NaN,0,38.807,-104.823,POINT (-104.82315 38.80666)
100,NaN,1/4/2020,COLORADO SPRINGS,EL PASO,2020,38.872175,-104.720847,POWERS BLVD N,CONSTITUTION AVE,Pedestrian,...,NaN,NaN,Pedestrian,Pedestrian,Pedestrian,NaN,0,38.872,-104.721,POINT (-104.72085 38.87218)
132,NaN,1/6/2020,COLORADO SPRINGS,EL PASO,2020,38.896742,-104.835750,GARDEN OF THE GODS RD W,CHESTNUT ST N,Pedestrian,...,NaN,NaN,Pedestrian,Pedestrian,NaN,Pedestrian,0,38.897,-104.836,POINT (-104.83575 38.89674)
174,NaN,1/7/2020,COLORADO SPRINGS,EL PASO,2020,39.797530,-104.742830,CHELTON RD S,ASTROZON BLVD,Pedestrian,...,NaN,NaN,Pedestrian,Pedestrian,NaN,Pedestrian,0,39.798,-104.743,POINT (-104.74283 39.79753)


In [45]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [46]:
# interactive Open Street Map map
gdf.explore()

In [47]:
# Create a color map based on the year range
year_min, year_max = gdf['Crash Year'].min(), gdf['Crash Year'].max()
norm = colors.Normalize(vmin=year_min, vmax=year_max)
cmap = cm.ScalarMappable(norm=norm, cmap='viridis')

# Get a central point for initial map focus
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()

# Create the base Folium map (you can change the tileset)
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

# Add each crash as a colored circle
for _, row in gdf.iterrows():
    color = colors.to_hex(cmap.to_rgba(row['Crash Year']))
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Year: {row['Crash Year']}"
    ).add_to(m)

# Add a color legend (branca colormap)
from branca.colormap import LinearColormap
colormap = LinearColormap(
    colors=[colors.to_hex(cmap.to_rgba(y)) for y in range(year_min, year_max + 1)],
    vmin=year_min, vmax=year_max,
    caption="Crash Year"
)
colormap.add_to(m)

m

# save map to an html file
m.save('cdot_crash_bikeped_2020_to_2024_by_year.html')

In [48]:
# Create a color map based on the year range
val_min, val_max = gdf['Number Killed'].min(), gdf['Number Killed'].max()
norm = colors.Normalize(vmin=val_min, vmax=val_max)
cmap = cm.ScalarMappable(norm=norm, cmap='viridis')

gdf_fatalities = gdf.loc[gdf['Number Killed'] > 0]

# Get a central point for initial map focus
center_lat = gdf_fatalities['Latitude'].mean()
center_lon = gdf_fatalities['Longitude'].mean()

# Create the base Folium map (you can change the tileset)
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

# Add each crash as a colored circle
for _, row in gdf_fatalities.iterrows():
    color = colors.to_hex(cmap.to_rgba(row['Number Killed']))
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Number Killed: {row['Number Killed']}"
    ).add_to(m)

# Add a color legend (branca colormap)
from branca.colormap import LinearColormap
colormap = LinearColormap(
    colors=[colors.to_hex(cmap.to_rgba(y)) for y in range(val_min, val_max + 1)],
    vmin=val_min, vmax=val_max,
    caption="Number Killed"
)
colormap.add_to(m)

m

In [49]:
# Heat Map
from folium.plugins import HeatMap

# center map on mean lat/long of data
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=8)
# m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# load heatmap data array
heat_data = [[row['Latitude'], row['Longitude']] for index, row in gdf.iterrows()]

# add heatmap layer to the map
HeatMap(heat_data).add_to(m)

m

### Plot Categorical Columns 

In [50]:
import geopandas as gpd
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from branca.element import Element

# ---- GeoDataFrame assumptions ----
# Columns: ['Latitude', 'Longitude', 'MHE']
# CRS: EPSG:4326 (WGS84)
# If not, reproject:
# gdf = gdf.to_crs(epsg=4326)

# ---- Assign colors for each MHE category ----
mhe_types = gdf['MHE'].unique()
colormap = cm.get_cmap('tab10', len(mhe_types))
color_dict = {t: mcolors.to_hex(colormap(i)) for i, t in enumerate(mhe_types)}

# ---- Create base map ----
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=9, tiles='CartoDB positron')

# ---- Add each crash point ----
for _, row in gdf.iterrows():
    mhe_type = row['MHE']
    color = color_dict[mhe_type]

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"MHE: {mhe_type}"
    ).add_to(m)

# ---- Add legend ----
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 200px;
    z-index:9999;
    background-color: white;
    border:2px solid grey;
    border-radius:8px;
    padding:10px;
    font-size:14px;
    box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
">
<b>MHE</b><br>
"""
for name, hexcol in color_dict.items():
    legend_html += (
        f'<div style="margin-top:6px;">'
        f'<span style="display:inline-block;width:14px;height:14px;background:{hexcol};margin-right:8px;vertical-align:middle;border:1px solid #444;"></span>'
        f'<span style="vertical-align:middle;">{name}</span>'
        f'</div>'
    )
legend_html += "</div>"

legend = Element(legend_html)
m.get_root().html.add_child(legend)

m

In [51]:
print(gdf["MHE"].unique())
mhe_to_simple = {
    'Pedestrian': "Pedestrian", 
    'Bicycle/Motorized Bicycle': "Bike", 
    'Curb': 'object', 
    'Other Non-Fixed Object (Describe in Narrative)': 'object', 
    'Front to Rear': 'car', 
    'Parked Motor Vehicle': 'car', 
    'Guardrail Face': 'object', 
    'School Age To/From School': 'School Age To/From School', 
    'Other Fixed Object (Describe in Narrative)': 'object',
    'Front to Side': 'car',
    'Fell from Motor Vehicle': 'driver',
    'Overturning/Rollover': 'car', 
    'Vehicle Debris or Cargo': 'object', 
    'Sign': 'object', 
    'Tree': 'object',
    'Overhead Structure (Not Bridge)': 'object', 
    'Front to Front': 'car', 
    'Light Pole/Utility Pole': 'object', 
    'Ground': 'object',
    'Side to Side-Same Direction': 'car'
}

gdf["MHE_simple"] = gdf["MHE"].map(mhe_to_simple)

['Pedestrian' 'Bicycle/Motorized Bicycle' 'Curb' nan
 'Other Non-Fixed Object (Describe in Narrative)' 'Front to Rear'
 'Parked Motor Vehicle' 'Guardrail Face' 'School Age To/From School'
 'Other Fixed Object (Describe in Narrative)' 'Front to Side'
 'Fell from Motor Vehicle' 'Overturning/Rollover'
 'Vehicle Debris or Cargo' 'Sign' 'Tree' 'Overhead Structure (Not Bridge)'
 'Front to Front' 'Light Pole/Utility Pole' 'Ground'
 'Side to Side-Same Direction']


In [54]:
import geopandas as gpd
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# ---- Assign colors for each MHE category ----
mhe_types = gdf['MHE_simple'].unique()
colormap = cm.get_cmap('tab10', len(mhe_types))
color_dict = {t: mcolors.to_hex(colormap(i)) for i, t in enumerate(mhe_types)}

# ---- Create base map ----
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=9, tiles='CartoDB positron')

# ---- Add each MHE type as its own layer ----
for mhe_type in mhe_types:
    subset = gdf[gdf['MHE_simple'] == mhe_type]
    color = color_dict[mhe_type]

    fg = folium.FeatureGroup(name=str(mhe_type))

    for _, row in subset.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"MHE: {mhe_type}"
        ).add_to(fg)

    fg.add_to(m)

# ---- Add layer control (interactive toggle) ----
folium.LayerControl(collapsed=False).add_to(m)

# ---- Add legend ----
from branca.element import Element

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 200px;
    z-index:9999;
    background-color: white;
    border:2px solid grey;
    border-radius:8px;
    padding:10px;
    font-size:14px;
    box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
">
<b>MHE</b><br>
"""
for name, hexcol in color_dict.items():
    legend_html += (
        f'<div style="margin-top:6px;">'
        f'<span style="display:inline-block;width:14px;height:14px;background:{hexcol};'
        f'margin-right:8px;vertical-align:middle;border:1px solid #444;"></span>'
        f'<span style="vertical-align:middle;">{name}</span>'
        f'</div>'
    )
legend_html += "</div>"

legend = Element(legend_html)
m.get_root().html.add_child(legend)

m

# save map to an html file
m.save('cdot_crash_bikeped_2020_to_2024_by_mhe_simplified.html')

### Set up H3 Grid

In [55]:
import geopandas as gpd
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from shapely.geometry import Polygon
import h3
from branca.colormap import LinearColormap

# Ensure CRS is EPSG:4326
gdf = gdf.to_crs(epsg=4326)

# ---- Choose H3 resolution ----
resolution = 9

# ---- Convert points to H3 hex index ----
gdf['h3_index'] = gdf.apply(
    lambda row: h3.latlng_to_cell(row['Latitude'], row['Longitude'], resolution),
    axis=1
)

# ---- Aggregate crash counts per H3 cell ----
counts = gdf.groupby('h3_index').size().reset_index(name='count')

# ---- Convert H3 cells to polygons ----
def h3_to_polygon(h):
    boundary = h3.cell_to_boundary(h)
    # Swap (lat, lon) → (lon, lat) for Shapely if needed
    return Polygon([(lon, lat) for lat, lon in boundary])

counts['geometry'] = counts['h3_index'].apply(h3_to_polygon)
counts_gdf = gpd.GeoDataFrame(counts, geometry='geometry', crs='EPSG:4326')

# ---- Create Folium base map ----
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=9, tiles='CartoDB positron')

# ---- Set up color mapping based on crash count ----
max_count = counts_gdf['count'].max()
colormap = cm.get_cmap('Reds')
norm = mcolors.Normalize(vmin=0, vmax=max_count)

# ---- Add polygons to map with proper lambda scoping ----
for _, row in counts_gdf.iterrows():
    color = mcolors.to_hex(colormap(norm(row['count'])))
    folium.GeoJson(
        data=row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.6
        },
        tooltip=folium.Tooltip(f"Crash Count: {row['count']}")
    ).add_to(m)

# ---- Add a legend ----
legend = LinearColormap(
    colors=[mcolors.to_hex(colormap(x)) for x in [0, 0.25, 0.5, 0.75, 1]],
    vmin=0, vmax=max_count,
    caption='Crash Count per H3 Cell'
)
legend.add_to(m)

# ---- Fit map to bounds of all hexes ----
bounds = counts_gdf.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# ---- Display map ----
m

# save map to an html file
m.save('cdot_crash_bikeped_2020_to_2024_by_crash_count_H3.html')